<a href="https://colab.research.google.com/github/anthonygtellez/GHSA-Analytics/blob/main/Github_Security_Advisory_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛡️ GitHub Security Advisory Database Analyzer

This project provides an interactive Google Colab notebook for analyzing the [GitHub Security Advisory Database](https://github.com/github/advisory-database), enriched with CISA's Known Exploited Vulnerabilities (KEV) catalog and NLP-based clustering for better threat identification.

---

## 🔍 What This Project Does

- Parses security advisories from GitHub's official advisory database
- Groups advisories by:
  - Severity (LOW, MODERATE, HIGH, CRITICAL)
  - Ecosystem (npm, PyPI, Maven, etc.)
  - Package name
- Tags advisories with Known Exploited Vulnerabilities (KEV) using CISA's CVE list
- Saves structured `.parquet` and `.json` files for analysis
- Visualizes risks using:
  - Interactive Plotly charts
  - UMAP-based 2D clustering of summaries
- Helps identify **vulnerable packages** and **shared attack vectors**

---

## 🚀 How to Use the Notebook

1. **Open in Google Colab**

   Upload the notebook or open it directly via:
   ```
   File > Upload Notebook > Select `CROGL_Github_Security_Advisory_Database.ipynb`
   ```

2. **Run all cells**

   From the Colab menu:
   ```
   Runtime > Run all
   ```

3. **Explore the Outputs**

   - Parquet files for each severity are saved in `parsed_advisories/`
   - Visual charts will render in-place
   - You can extend analysis using DataFrames for your own security tools

---

## 🔬 Requirements (Handled Automatically in Colab)

- `umap-learn`
- `sentence-transformers`
- `plotly`
- `pandas`
- `huggingface_hub` (optional for login)

---

## 🔐 Hugging Face Access (Optional)

Some NLP models require Hugging Face authentication.

### Option 1: Login from notebook
```python
from huggingface_hub import login
login()
```

### Option 2: Use Colab Secret Manager
Store your token as `HF_TOKEN`:
```python
import os
from huggingface_hub import login
login(token=os.environ["HF_TOKEN"])
```

---

## 📊 Visualization Features

- Grouped by ecosystem, severity, and KEV status
- UMAP-based clustering of CVE summary descriptions
- Identify similar attack vectors across packages

---

## 🤝 Credits

- GitHub [advisory-database](https://github.com/github/advisory-database)
- CISA [Known Exploited Vulnerabilities Catalog](https://www.cisa.gov/known-exploited-vulnerabilities-catalog)
- Hugging Face `sentence-transformers`
- Plotly + UMAP for clustering

---

## 📄 License

MIT License — feel free to modify and adapt this notebook for your security team or research! Happy Hunting @anthonygtellez

---


# 🧱 Step 1: Setup

In [11]:
!git clone --depth=1 https://github.com/github/advisory-database.git
!pip install pyyaml pandas pyarrow

fatal: destination path 'advisory-database' already exists and is not an empty directory.


In [12]:
import os
import glob
import json
import requests
import pandas as pd
from collections import defaultdict

# ✅ Enable interactive DataFrames in Google Colab
from google.colab import data_table
data_table.enable_dataframe_formatter()

# 🗂️ Step 2: Paths and KEV Loading

In [13]:
advisory_dir = "advisory-database/advisories"
ecosystems = os.listdir(advisory_dir)

# Load CISA KEV List
kev_url = "https://www.cisa.gov/sites/default/files/feeds/known_exploited_vulnerabilities.json"
kev_data = requests.get(kev_url).json()
kev_cves = {item['cveID'] for item in kev_data['vulnerabilities']}

# Traverse GitHub-reviewed advisories

In [14]:
import os
import json
import copy
from collections import defaultdict

advisory_dir = "/content/advisory-database/advisories/github-reviewed"
output = defaultdict(lambda: defaultdict(list))
valid_severities = {"LOW", "MODERATE", "HIGH", "CRITICAL"}

count_total = 0
count_loaded = 0
count_grouped = 0

for year in os.listdir(advisory_dir):
    year_path = os.path.join(advisory_dir, year)
    if not os.path.isdir(year_path): continue

    for month in os.listdir(year_path):
        month_path = os.path.join(year_path, month)
        if not os.path.isdir(month_path): continue

        for ghsa_dir in os.listdir(month_path):
            ghsa_path = os.path.join(month_path, ghsa_dir)
            json_file = os.path.join(ghsa_path, f"{ghsa_dir}.json")
            count_total += 1

            if os.path.isfile(json_file):
                try:
                    with open(json_file, 'r') as f:
                        data = json.load(f)
                    count_loaded += 1

                    # Grab severity from nested field
                    severity = data.get('database_specific', {}).get('severity', 'UNKNOWN').upper()
                    if severity not in valid_severities:
                        continue

                    # KEV tagging based on CVEs
                    cve_ids = [aid for aid in data.get('aliases', []) if aid.startswith("CVE-")]
                    is_kev = any(cve in kev_cves for cve in cve_ids)

                    # Each affected package = one enriched advisory entry
                    for a in data.get('affected', []):
                        pkg = a.get('package', {})
                        ecosystem = pkg.get('ecosystem', 'unknown') or 'unknown'
                        pkg_name = pkg.get('name', 'unknown') or 'unknown'

                        # Deep copy full advisory and add enriched fields
                        enriched = copy.deepcopy(data)
                        enriched['ecosystem'] = ecosystem
                        enriched['package_name'] = pkg_name
                        enriched['severity'] = severity
                        enriched['is_kev'] = is_kev

                        output[severity][ecosystem].append(enriched)
                        count_grouped += 1

                except Exception as e:
                    print(f"⚠️ Failed to load {json_file}: {e}")

# ✅ Summary stats

In [15]:
print(f"📁 Found total advisory folders: {count_total}")
print(f"✅ Successfully loaded .json files: {count_loaded}")
print(f"📊 Advisories grouped into output: {count_grouped}")
print(f"🔎 Example severities: {list(output.keys())[:3]}")

📁 Found total advisory folders: 21942
✅ Successfully loaded .json files: 21942
📊 Advisories grouped into output: 39884
🔎 Example severities: ['HIGH', 'CRITICAL', 'MODERATE']


# 💾 Step 4: Save as JSON and Parquet

In [16]:
import pandas as pd
import os
import copy

# Output folder
os.makedirs("parsed_advisories", exist_ok=True)

# Flat export structure
flat_output = {'LOW': [], 'MODERATE': [], 'HIGH': [], 'CRITICAL': []}

# Flatten and enrich each advisory entry
for severity, eco_groups in output.items():
    for ecosystem, advisories in eco_groups.items():
        for advisory in advisories:
            enriched = copy.deepcopy(advisory)  # clone to avoid overwriting original

            # Enrich with contextual metadata
            enriched['ecosystem'] = advisory.get('ecosystem', ecosystem)
            enriched['package_name'] = advisory.get('package_name', 'unknown')
            enriched['severity'] = advisory.get('severity', severity)
            enriched['is_kev'] = advisory.get('is_kev', False)

            flat_output[severity].append(enriched)

# Save as JSON (preserve full structure) and Parquet (flattened)
for level, entries in flat_output.items():
    # ✅ JSON — keep full nested structure
    json_path = f"parsed_advisories/{level.lower()}_advisories.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(entries, f, indent=2)

    # ✅ Parquet — flatten lists for dataframe compatibility
    df = pd.json_normalize(entries)

    # Optional: flatten CVE list to comma-separated string for easier use in analytics
    if 'cves' in df.columns:
        df['cves'] = df['cves'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

    parquet_path = f"parsed_advisories/{level.lower()}_advisories.parquet"
    df.to_parquet(parquet_path, index=False)

    print(f"✅ Saved {len(df)} advisories to:\n  - {json_path}\n  - {parquet_path}")

✅ Saved 3158 advisories to:
  - parsed_advisories/low_advisories.json
  - parsed_advisories/low_advisories.parquet
✅ Saved 18183 advisories to:
  - parsed_advisories/moderate_advisories.json
  - parsed_advisories/moderate_advisories.parquet
✅ Saved 13638 advisories to:
  - parsed_advisories/high_advisories.json
  - parsed_advisories/high_advisories.parquet
✅ Saved 4905 advisories to:
  - parsed_advisories/critical_advisories.json
  - parsed_advisories/critical_advisories.parquet


## 📊 Visualizer for Parquet Advisory Files

In [17]:
import pandas as pd
import os
from IPython.display import display

# 🔍 Directory where Parquet files are stored
parquet_dir = "parsed_advisories"
severities = ['low', 'moderate', 'high', 'critical']

# 🔁 Load and display only key columns
for level in severities:
    file_path = os.path.join(parquet_dir, f"{level}_advisories.parquet")

    if os.path.exists(file_path):
        df = pd.read_parquet(file_path)

        # Select only UMAP-related fields
        cols_to_show = ['id', 'summary', 'severity', 'ecosystem', 'package_name', 'is_kev']
        display_df = df[[col for col in cols_to_show if col in df.columns]]

        print(f"\n📁 Displaying {level.capitalize()} Severity Advisories ({len(display_df)} entries)")
        display(display_df.head(20))  # Optional: limit preview for speed
    else:
        print(f"⚠️ File not found: {file_path}")


📁 Displaying Low Severity Advisories (3158 entries)


,id,summary,severity,ecosystem,package_name,is_kev
0,GHSA-vxf6-w9mp-95hm,Puppet supports use of IP addresses in certnam...,LOW,RubyGems,puppet,False
1,GHSA-fqrr-rrwg-69pv,Local API Login Credentials Disclosure in para...,LOW,RubyGems,paratrooper-pingdom,False
2,GHSA-p692-7mm3-3fxg,actionpack is vulnerable to remote bypass auth...,LOW,RubyGems,actionpack,False
3,GHSA-p692-7mm3-3fxg,actionpack is vulnerable to remote bypass auth...,LOW,RubyGems,actionpack,False
4,GHSA-8jxj-9r5f-w3m2,Puppet allows local users to obtain sensitive ...,LOW,RubyGems,puppet,False
5,GHSA-959j-5g9v-3fpq,Paratrooper-newrelic Exposes of Sensitive Info...,LOW,RubyGems,paratrooper-newrelic,False
6,GHSA-c5qq-g673-5p49,Puppet allows local users to overwrite arbitra...,LOW,RubyGems,puppet,False
7,GHSA-g89m-3wjw-h857,Puppet vulnerable to Path Traversal,LOW,RubyGems,puppet,False
8,GHSA-g89m-3wjw-h857,Puppet vulnerable to Path Traversal,LOW,RubyGems,puppet,False
9,GHSA-mx9f-w8qq-q5jf,rest-client allows local users to obtain sensi...,LOW,RubyGems,rest-client,False



📁 Displaying Moderate Severity Advisories (18183 entries)


,id,summary,severity,ecosystem,package_name,is_kev
0,GHSA-h9mj-fghc-664w,Denial of Service in mqtt,MODERATE,npm,mqtt,False
1,GHSA-rh6c-q938-3r9q,Moderate severity vulnerability that affects v...,MODERATE,npm,validator,False
2,GHSA-9959-c6q6-6qp3,Moderate severity vulnerability that affects v...,MODERATE,npm,validator,False
3,GHSA-q4qq-fm7q-cwp5,Multiple XSS Filter Bypasses in validator,MODERATE,npm,validator,False
4,GHSA-hpcf-8vf9-q4gj,jQuery-UI vulnerable to Cross-site Scripting i...,MODERATE,npm,jquery-ui,False
5,GHSA-87vv-r9j6-g5qv,Regular Expression Denial of Service in moment,MODERATE,npm,moment,False
6,GHSA-wcm2-9c89-wmfm,Cross-site Scripting in jquery-ui,MODERATE,npm,jquery-ui,False
7,GHSA-fmr4-7g9q-7hc7,Moderate severity vulnerability that affects h...,MODERATE,npm,handlebars,False
8,GHSA-jgqf-hwc5-hh37,Root Path Disclosure in send,MODERATE,npm,send,False
9,GHSA-cfjh-p3g4-3q2f,VBScript Content Injection in marked,MODERATE,npm,marked,False



📁 Displaying High Severity Advisories (13638 entries)


,id,summary,severity,ecosystem,package_name,is_kev
0,GHSA-3rpr-mg43-xhq4,auth0-js Privilege Escalation Vulnerability,HIGH,npm,auth0-js,False
1,GHSA-pm9p-9926-w68m,Denial of Service in ecstatic,HIGH,npm,ecstatic,False
2,GHSA-3f5c-4qxj-vmpf,Next.js Directory Traversal Vulnerability,HIGH,npm,next,False
3,GHSA-7fpw-cfc4-3p2c,Duplicate advisory: High severity vulnerabilit...,HIGH,npm,passport-wsfed-saml2,False
4,GHSA-jjv7-qpx3-h62q,Denial-of-Service Memory Exhaustion in qs,HIGH,npm,qs,False
5,GHSA-x6fg-f45m-jf5q,Regular Expression Denial of Service in semver,HIGH,npm,semver,False
6,GHSA-c9f4-xj24-8jqx,Regular Expression Denial of Service in uglify-js,HIGH,npm,uglify-js,False
7,GHSA-w3w8-37jv-2c58,Cross-Site Scripting in mustache,HIGH,npm,mustache,False
8,GHSA-cqr7-78pj-3g7j,File Descriptor Leak Can Cause DoS Vulnerabili...,HIGH,npm,hapi,False
9,GHSA-f522-ffg8-j8r6,Regular Expression Denial of Service in is-my-...,HIGH,npm,is-my-json-valid,False



📁 Displaying Critical Severity Advisories (4905 entries)


,id,summary,severity,ecosystem,package_name,is_kev
0,GHSA-vx5c-87qx-cv6c,Arbitrary Code Execution in mathjs,CRITICAL,npm,mathjs,False
1,GHSA-pv8x-p9hq-j328,Arbitrary Code Execution in mathjs,CRITICAL,npm,mathjs,False
2,GHSA-xxvw-45rp-3mj2,Deserialization Code Execution in js-yaml,CRITICAL,npm,js-yaml,False
3,GHSA-34r7-q49f-h37c,Incorrect Handling of Non-Boolean Comparisons ...,CRITICAL,npm,uglify-js,False
4,GHSA-q5pq-pgrv-fh89,dns-sync command injection vulnerability,CRITICAL,npm,dns-sync,False
5,GHSA-5j3g-jfq3-7jwx,Arbitrary JavaScript Execution in bassmaster,CRITICAL,npm,bassmaster,False
6,GHSA-5c8j-xr24-2665,Potential Command Injection in printer,CRITICAL,npm,printer,False
7,GHSA-3w5v-p54c-f74x,ejs is vulnerable to remote code execution due...,CRITICAL,npm,ejs,False
8,GHSA-mw35-24gh-f82w,keycloak-connect and keycloak-js improperly ha...,CRITICAL,npm,keycloak-connect,False
9,GHSA-mw35-24gh-f82w,keycloak-connect and keycloak-js improperly ha...,CRITICAL,npm,keycloak-js,False


# 🧪 Install required libraries for UMAP Analysis

In [18]:
!pip install -q umap-learn sentence-transformers plotly

In [19]:
import os
import pandas as pd
import plotly.express as px
import umap
from sentence_transformers import SentenceTransformer

# 🔁 Load all Parquet files
parquet_dir = "parsed_advisories"
severities = ['low', 'moderate', 'high', 'critical']
df_list = []

for level in severities:
    path = os.path.join(parquet_dir, f"{level}_advisories.parquet")
    if os.path.exists(path):
        df = pd.read_parquet(path)
        df['severity'] = level.upper()
        df_list.append(df)

df = pd.concat(df_list, ignore_index=True)

# 🧹 Clean and prep
df['ecosystem'] = df['ecosystem'].fillna('unknown')
df['package_name'] = df['package_name'].fillna('unknown')
df['summary'] = df['summary'].fillna('')
df['is_kev'] = df['is_kev'].fillna(False)

# 🎯 Group by package + ecosystem and summarize
grouped = (
    df.groupby(['ecosystem', 'package_name'])
    .agg({
        'summary': lambda x: ' '.join(x),
        'severity': lambda x: x.value_counts().idxmax(),
        'is_kev': 'max',
        'id': 'count'
    })
    .reset_index()
    .rename(columns={'id': 'advisory_count'})
)

# 🔡 Map severity to numerical scale
severity_map = {'LOW': 0, 'MODERATE': 1, 'HIGH': 2, 'CRITICAL': 3}
grouped['severity_score'] = grouped['severity'].map(severity_map)

# 🔤 Sentence embeddings from NLP summaries
model = SentenceTransformer('all-MiniLM-L6-v2')
summary_embeddings = model.encode(grouped['summary'].tolist(), show_progress_bar=True)

# 🔽 Dimensionality reduction with UMAP
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine', random_state=42)
embedding = umap_model.fit_transform(summary_embeddings)

grouped['umap_x'] = embedding[:, 0]
grouped['umap_y'] = embedding[:, 1]

Batches:   0%|          | 0/297 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



# 📊 Plot clusters

In [20]:
fig = px.scatter(
    grouped,
    x='umap_x',
    y='umap_y',
    color='severity',
    size='advisory_count',
    symbol='is_kev',
    hover_name='package_name',
    hover_data=['ecosystem', 'advisory_count', 'severity', 'is_kev'],
    title='📌 UMAP Clustering of Risky Software Packages (by Summary + Severity)',
    height=750
)
fig.update_traces(marker=dict(line=dict(width=0.5, color='DarkSlateGrey')))
fig.update_layout(legend_title='Severity')
fig.show()

## 🛠️ Troubleshooting: Hugging Face Authentication in Google Colab

If you're running this in Colab and see an error like:

```
OSError: We couldn't connect to 'https://huggingface.co' to load this model, and it looks like all-MiniLM-L6-v2 is not the path to a directory containing a model.
```

This means Hugging Face requires authentication to use `sentence-transformers`.

### ✅ To fix:
1. Get a token from https://huggingface.co/settings/tokens
2. In Colab, run:
```python
from huggingface_hub import login
login()
```
3. Paste in your token when prompted.

You only need to do this once per session to allow access to pretrained models from Hugging Face.


### ✅ Option 2: Use Google Colab’s Secrets Manager
1.	In the left sidebar, click the key icon or go to ⋮ More > Secrets.
2.	Add a new secret with the name HF_TOKEN and paste your Hugging Face token as the value.
3.	Then in your notebook, run:

```python
import os
from huggingface_hub import login
login(token=os.environ["HF_TOKEN"])
```